In [1]:
from uncertainties import ufloat
from uncertainties.umath import log10
import pandas as pd
from typing import Tuple
from extinction import extinction_coefficient, gaia_g_extinction, gaia_B_RP_extinction, color, absolute_magnitude

Configuration file not found:

    /home/mjablons/.dustmapsrc

To create a new configuration file in the default location, run the following python code:

    from dustmaps.config import config
    config.reset()

Note that this will delete your configuration! For example, if you have specified a data directory, then dustmaps will forget about its location.


In [2]:
c = pd.read_csv('./data/ngc_188/dbscan_clustered.csv').drop(columns='Unnamed: 0')

In [12]:
c['phot_bp_mean_mag_err'] = 2.5/(np.log(10)*c['phot_bp_mean_flux_over_error'])
c['phot_rp_mean_mag_err'] = 2.5/(np.log(10)*c['phot_rp_mean_flux_over_error'])
c['phot_g_mean_mag_err'] = 2.5/(np.log(10)*c['phot_g_mean_flux_over_error'])

In [13]:
c['E_V'] = extinction_coefficient(c.ra.values, c.dec.values, c.parallax.values)
c[['E(BP-RP)', 'reddening_err']] = gaia_B_RP_extinction(c['E_V'].values)
c[['E_G', 'E_G_err']] = gaia_g_extinction(c['E_V'].values)

/home/mjablons/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/home/mjablons/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [14]:
c.head()

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,libname_gspphot,label,E_V,E(BP-RP),reddening_err,E_G,E_G_err,phot_bp_mean_mag_err,phot_rp_mean_mag_err,phot_g_mean_mag_err
0,1636148068921376768,Gaia DR3 573339170073986432,573339170073986432,1183470769,2016.0,17.141358,0.117548,84.451083,0.113088,0.547640,...,PHOENIX,0,0.21,0.474470,0.2361,1.014187,0.08553,0.017684,0.015375,0.001813
1,1636148068921376768,Gaia DR3 573341304674009856,573341304674009856,297753663,2016.0,17.627554,0.041246,84.519049,0.038805,0.554564,...,MARCS,0,0.21,0.474470,0.2361,1.014187,0.08553,0.006012,0.004857,0.000605
2,1636148068921376768,Gaia DR3 573343052724431360,573343052724431360,654512527,2016.0,15.764065,0.168689,84.508904,0.166331,0.560219,...,NaN,0,0.21,0.474470,0.2361,1.014187,0.08553,0.073426,0.017784,0.002152
3,1636148068921376768,Gaia DR3 573344633272421120,573344633272421120,1350517326,2016.0,15.401420,0.238333,84.585590,0.231550,0.465927,...,NaN,0,0.14,0.378539,0.2361,0.861766,0.08553,0.047550,0.053979,0.003215
4,1636148068921376768,Gaia DR3 573351475155035008,573351475155035008,236544348,2016.0,13.267578,0.474411,84.456973,0.483350,0.519350,...,NaN,0,0.15,0.392306,0.2361,0.883490,0.08553,0.158200,0.062628,0.005841


In [19]:
c[['color', 'color_err']] = 

AttributeError: 'numpy.ndarray' object has no attribute 'strip'

In [11]:
c.iloc[0].parallax

0.547640404087395

In [12]:
c.iloc[0].parallax_error

0.12622535

In [25]:
def absolute_magnitude(g_mean_mag: np.float32,
                       g_mean_mag_err: np.float32,
                       g_ext: np.float32,
                       g_ext_err: np.float32,
                       parallax: np.float32,
                       parallax_err: np.float32) -> Tuple[np.float32, np.float32]:
    value = (ufloat(g_mean_mag, g_mean_mag_err)+
             5*log10(ufloat(parallax, parallax_err))-10-ufloat(g_ext, g_ext_err))
    return value.nominal_value, value.std_dev

In [26]:
def color(bp_mean_mag: np.float32,
          bp_mean_mag_err: np.float32,
          rp_mean_mag: np.float32,
          rp_mean_mag_err: np.float32,
          reddening: np.float32,
          reddening_err: np.float32) -> Tuple[np.float32, np.float32]:
    value = (ufloat(bp_mean_mag, bp_mean_mag_err)-ufloat(rp_mean_mag, rp_mean_mag_err)-
             ufloat(reddening, reddening_err))
    return value.nominal_value, value.std_dev